In [24]:
import pandas as pd
import plotly as pt
import folium
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
confirmed_cases = pd.read_csv('time_series_covid19_confirmed_global.csv')
# print(confirmed_cases.head(15))

country_coordinates = {}

for index,row in confirmed_cases.iterrows():
    if row['Country/Region'] in country_coordinates:
        pass
    else:
        country_coordinates[row['Country/Region']] = (row.Lat, row.Long)

country_coordinates


{'Afghanistan': (33.93911, 67.709953),
 'Albania': (41.1533, 20.1683),
 'Algeria': (28.0339, 1.6596),
 'Andorra': (42.5063, 1.5218),
 'Angola': (-11.2027, 17.8739),
 'Antarctica': (-71.9499, 23.347),
 'Antigua and Barbuda': (17.0608, -61.7964),
 'Argentina': (-38.4161, -63.6167),
 'Armenia': (40.0691, 45.0382),
 'Australia': (-35.4735, 149.0124),
 'Austria': (47.5162, 14.5501),
 'Azerbaijan': (40.1431, 47.5769),
 'Bahamas': (25.025885, -78.035889),
 'Bahrain': (26.0275, 50.55),
 'Bangladesh': (23.685, 90.3563),
 'Barbados': (13.1939, -59.5432),
 'Belarus': (53.7098, 27.9534),
 'Belgium': (50.8333, 4.469936),
 'Belize': (17.1899, -88.4976),
 'Benin': (9.3077, 2.3158),
 'Bhutan': (27.5142, 90.4336),
 'Bolivia': (-16.2902, -63.5887),
 'Bosnia and Herzegovina': (43.9159, 17.6791),
 'Botswana': (-22.3285, 24.6849),
 'Brazil': (-14.235, -51.9253),
 'Brunei': (4.5353, 114.7277),
 'Bulgaria': (42.7339, 25.4858),
 'Burkina Faso': (12.2383, -1.5616),
 'Burma': (21.9162, 95.956),
 'Burundi': (-3.

In [26]:
confirmed_cases_by_country = confirmed_cases.groupby("Country/Region").sum(numeric_only=True)\
                                            .drop(columns=['Lat', 'Long'])
confirmed_cases_by_country.head(15)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
Albania,0,0,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
Algeria,0,0,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
Andorra,0,0,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
Angola,0,0,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288
Antarctica,0,0,0,0,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,9106,9106,9106,9106,9106,9106,9106,9106,9106,9106
Argentina,0,0,0,0,0,0,0,0,0,0,...,10044125,10044125,10044125,10044125,10044125,10044125,10044957,10044957,10044957,10044957
Armenia,0,0,0,0,0,0,0,0,0,0,...,446819,446819,446819,446819,446819,446819,446819,446819,447308,447308


In [27]:
list(confirmed_cases_by_country.columns)
def date_converter(date_str):
    month_map = {
    '1': "January",
    '2': "February",
    '3': "March",
    '4': "April",
    '5': "May",
    '6': "June",
    '7': "July",
    '8': "August",
    '9': "September",
    '10': "October",
    '11': "November",
    '12': "December"
    }

    split_date = date_str.split('/')

    return month_map[split_date[0]] + split_date[-1]

In [28]:
month_list = []
for i in confirmed_cases_by_country.columns:
    month_list.append(date_converter(i))

confirmed_cases_by_country.columns = month_list

month_set = set()
for i in month_list:
    month_set.add(i)


In [29]:
monthly_data_df = pd.DataFrame()
monthly_data_df

""


In [30]:
month_set
# for col in columns:

{'April20',
 'April21',
 'April22',
 'August20',
 'August21',
 'August22',
 'December20',
 'December21',
 'December22',
 'February20',
 'February21',
 'February22',
 'February23',
 'January20',
 'January21',
 'January22',
 'January23',
 'July20',
 'July21',
 'July22',
 'June20',
 'June21',
 'June22',
 'March20',
 'March21',
 'March22',
 'March23',
 'May20',
 'May21',
 'May22',
 'November20',
 'November21',
 'November22',
 'October20',
 'October21',
 'October22',
 'September20',
 'September21',
 'September22'}

In [13]:
confirmed_cases_by_country.head()

,January20,January20,January20,January20,January20,January20,January20,January20,January20,January20,...,February23,March23,March23,March23,March23,March23,March23,March23,March23,March23
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
Albania,0,0,0,0,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
Algeria,0,0,0,0,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
Andorra,0,0,0,0,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
Angola,0,0,0,0,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [11]:
nigerian_center=[9.0778,8.6775]
globe = folium.Map(zoom_start=3)
display(globe)